In [3]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

"\nChatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.\n\nImagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.\n\nCreate a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.\n\nThe final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.\n\nExample\nA: Hello MedAssistant.\nB: Hello. How can I help you?\nA: I don't feel well.\nB: Do you have any symptoms?\nA: I have cough and nausea.\nB: Do you want to book an appointment?\nA: Yes, please, for tomorrow.\n\n\nInfo:\n- Feel free to change or arrange a new dataset of intents\n- Try experimenting and tunin

In [4]:
import numpy as np
import torch
import json
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
#import nltk
#nltk.download('punkt')
from model import NeuralNet

# STEP 0: find intents patterns

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))

print('WORDS \n', all_words)
print('TAGS \n', tags)
print('PATTERNS \n', patterns)
print(len(all_words))

# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns

WORDS 
 ['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Good', 'evening', 'Good', 'night', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'hamburgers', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'courses', 'are', 'there', '?', 'What', 'do', 'you', 'cook', '?', 'Can', 'I', 'order', 'a', 'coke', '?', 'I', "'m", 'hungry', 'We', "'re", 'starving', 'I', "'d", 'like', 'to', 'have', 'a', 'pizza', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Apple', 'Pay', 'or', 'Paypal', '?', 'Can', 'I', 'pay', 'with', 'cash', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Let', 'me', 'laugh', '!']
TAGS 
 ['greeting', 'goodbye', 'thank

In [5]:
# STEP 1: Pre-process of the input
# lower case? stemming? stopwords?
ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(patterns), "patterns")
print(len(tags), "tags")
print(len(all_words), "unique stemmed words")

32 patterns
7 tags
70 unique stemmed words


In [6]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)
print(all_words)
X_train = np.array(X_train)
y_train = np.array(y_train)

["'d", "'m", "'re", "'s", 'a', 'accept', 'anyon', 'appl', 'are', 'bye', 'can', 'card', 'cash', 'coke', 'cook', 'cours', 'credit', 'day', 'deliveri', 'do', 'doe', 'even', 'funni', 'get', 'good', 'goodby', 'hamburg', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'hungri', 'i', 'is', 'joke', 'kind', 'later', 'laugh', 'let', 'like', 'long', 'lot', 'mastercard', 'me', 'my', 'night', 'of', 'or', 'order', 'pay', 'paypal', 'pizza', 'see', 'ship', 'someth', 'starv', 'take', 'tell', 'thank', 'that', 'there', 'to', 'we', 'what', 'when', 'which', 'with', 'you']


In [7]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience

num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss:{loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

# STEP 5: Save the model
file = "trained.pth"
torch.save(data, file)
print(f'training complete. file saved!')

C:\Users\harsh\anaconda3\envs\NLP\lib\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch [100/1000], Loss:0.8095
Epoch [200/1000], Loss:0.2459
Epoch [300/1000], Loss:0.0703
Epoch [400/1000], Loss:0.0106
Epoch [500/1000], Loss:0.0032
Epoch [600/1000], Loss:0.0035
Epoch [700/1000], Loss:0.0025
Epoch [800/1000], Loss:0.0009
Epoch [900/1000], Loss:0.0004
Epoch [1000/1000], Loss:0.0004
final loss: 0.0004
training complete. file saved!


In [8]:
import random

In [9]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the intents file
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

# Retrieve the model and all the sizings
file = "trained.pth"
data = torch.load(file)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# build the NN
model = NeuralNet(input_size, hidden_size, output_size).to(device)

model.load_state_dict(model_state)
model.eval()

# prepare a command-line conversation (don't forget something to make the user exit the script!)
bot_name = "Bixby"
print("Lets's have a chat! (type 'exit' to stop chatting!)")

while True:
    sentence = input("You: ")
    if sentence == "exit":
        print("Thanks for joining me!")
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)

    output = model(X)
    _, predicted = torch.max(output, dim = 1)
    tag = tags[predicted.item()]
    probs = torch.softmax(output, dim= 1)
    prob = probs[0][predicted.item()]
    print(prob)
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    
    else:
        print(f'{bot_name}: I do not understand. Try to be more specific')
    

Lets's have a chat! (type 'exit' to stop chatting!)
tensor(0.9994, grad_fn=<SelectBackward0>)
Bixby: We accept most major credit cards, and Paypal
tensor(0.9989, grad_fn=<SelectBackward0>)
Bixby: Hello, thanks for visiting
tensor(0.9996, grad_fn=<SelectBackward0>)
Bixby: Why do bees have sticky hair? Because they use honeycombs
tensor(0.9979, grad_fn=<SelectBackward0>)
Bixby: May we offer you a slice of hot pizza?
tensor(0.8294, grad_fn=<SelectBackward0>)
Bixby: Hey!
tensor(0.9989, grad_fn=<SelectBackward0>)
Bixby: What's up?
tensor(0.5632, grad_fn=<SelectBackward0>)
Bixby: I do not understand. Try to be more specific
Thanks for joining me!
